In [1]:
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import os
from img2vec_pytorch import Img2Vec
from PIL import Image
import xml.etree.ElementTree as ET

C:\Users\alexa\AppData\Roaming\Python\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\alexa\AppData\Roaming\Python\Python310\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
elastic_client = Elasticsearch(hosts=['http://localhost:9200'],
                               basic_auth=('elastic', 'master'))
model = Img2Vec()

In [3]:
def generate_embedding(filename):
    img = Image.open(filename).convert("RGB")
    vec = model.get_vec(img)
    return vec.tolist()

In [4]:
def build_doc(no:str, name:str, classes, vector):
  document={ 
    "no": no,
    "name": name,
    "classes": classes, 
    "embedding": vector
  }   
  return document

In [5]:
def get_files(path:str, extension:str):
    file_list = [f for f in os.listdir(path) if f.endswith(extension)]
    return file_list

In [6]:
def get_classes(path:str, file_name:str):
    tree = ET.parse(path+file_name)
    root = tree.getroot()
    classes=[]
    for child in root:
        if (child.tag == 'object'):
            for new_child in child:
                if (new_child.tag == 'name'):
                    classes.append(new_child.text)
    return classes

In [11]:
dataset_path = "./datasets/sb/images/"
index_name = "building_standard_index_new"

In [12]:
id = 0
actions = []
file_list = get_files(dataset_path, ".jpg")
actions = []
for file in file_list: 
    id += 1
    image_file = dataset_path+file
    #xml_file = file[0:len(file)-4]+".xml"
    classes = "teste"
    vector = generate_embedding(image_file)
    actions.append({"_index":index_name.lower(), 
                    "_source": build_doc(id, file, classes, vector)})
    if ((id % 50 ) == 0):
      bulk(elastic_client, actions)
      actions.clear()
      print("{} images were indexed!!!".format(id))

if len(actions) != 0:
  bulk(elastic_client, actions)
  actions.clear()
  print("{} images were indexed!!!".format(id))
print("Indexing was finished!!!")


10 images were indexed!!!
Indexing was finished!!!
